# requirements for graph subsystem
1. provides a graph, which is a collection of vertices and edges
1. the system must allow multiple graphs to be created
1. allow comparison 2 graphs for equality
1. graph must have a root vertex
1. graps, vertices, and edges must all have unique identifier
1. a complete graph must be fully connected ie must be able to traverse from the root vertex to any other vertex in the graph
    - a complete graph is defined as having only one vertex (the root) with no incoming edges
1. for now, vertex and edge properties are not required
1. a graph must allow vertices and edges to be added
1. vertices
    - vertices must have a type
1. edges
    - connects 2 vertices
    - has a from vertex and a to vertex
    - store the type of a relationship eg parent/child
1. remove vertices and edges
    - vertex must have 0 or 1 children
    - edges must have the same type
    - if 0 children:
        - vertex and incoming edge are removed
    - if 1 child:
        - new edge between child and grandparent of same type of edge between parent/child
        - parent and its incoming/outgoing edges removed

# requirements for TREE problem

1. vertex type is an integer (1+)
1. edge type is only 1 - parent/child

# implementation choices

1. edge type is an integer with a constant value of 0 representing parent/child
1. vertex types are integers with value 1+

Edge Class

In [2]:
class Edge:
    def __init__(self, id:int, fromVertex, toVertex):
        self.id = id
        self.edgeType = 0
        self.fromVertex = fromVertex
        self.toVertex = toVertex

    def vertexFrom(self):
        return self.fromVertex
    
    def vertexTo(self):
        return self.toVertex

Vertex Class

In [3]:
class Vertex:
    def __init__(self, id:int, vertexType:int):
        self.id = id
        self.vertexType = vertexType
        self.__incomingEdges = []
        self.__outgoingEdges = []

    def addEdge(self, edge:Edge):
        if edge.fromVertex == self:
            self.__outgoingEdges.append(edge)
        else:
            self.__incomingEdges.append(edge)

    def removeEdge(self, edge:Edge):
        if edge in self.__incomingEdges:
            self.__incomingEdges.remove(edge)
        elif edge in self.__outgoingEdges:
            self.__outgoingEdges.remove(edge)

    def isRoot(self):
        if self.__incomingEdges == 0:
            return True
        else:
            return False
        
    def childCount(self):
        return len(self.__outgoingEdges)
    
    def incomingEdges(self):
        return self.__incomingEdges
    
    def outgoingEdges(self):
        return self.__outgoingEdges
    
    def parent(self):
        if len(self.__incomingEdges) == 1:
            return self.__incomingEdges[0].vertexFrom()

    def child(self):
        if len(self.__outgoingEdges) == 1:
            return self.__outgoingEdges[0].vertexTo()

    def printVertex(self):
        print(f'vertex id = {self.id}, type = {self.vertexType}, incoming = {len(self.__incomingEdges)}, outgoing = {len(self.__outgoingEdges)}')

Graph class - vertex and edge management happens here

In [4]:
class Graph:
    def __init__(self, id:int):
        self.id = id
        self.__vertexCounter = 1
        self.__edgeCounter = 1
        self.__vertices = []
        self.__edges = []

    def __resetGraph(self):
        self.__vertexCounter = 1
        self.__edgeCounter = 1
        self.__vertices.clear()
        self.__edges.clear()

    def getRootVertext(self):
        return self.__vertices[0]

    def createRoot(self, vertexType:int):
        self.__resetGraph()
        self.__vertices.append(Vertex(self.__vertexCounter,vertexType))
        self.__vertexCounter += 1
        return self.__vertices[0]
    
    def createChildVertex(self, parentVertex:Vertex, childVertexType:int):
        childVertex = Vertex(self.__vertexCounter, childVertexType)
        edge = Edge(self.__edgeCounter, parentVertex, childVertex)
        parentVertex.addEdge(edge)
        childVertex.addEdge(edge)
        self.__vertexCounter += 1
        
        self.__vertices.append(childVertex)
        self.__edges.append(edge)
        return childVertex
    
    def removeVertex(self, vertexToRemove:Vertex):
        if vertexToRemove.isRoot():
            self.__resetGraph()
        elif vertexToRemove.childCount() < 2:
            if vertexToRemove.childCount() == 1:
                parent = vertexToRemove.parent()
                child = vertexToRemove.child()

                edge = Edge(self.__edgeCounter, parent, child)
                self.__edgeCounter += 1

                edgeToRemove = vertexToRemove.outgoingEdges()[0]
                self.__edges.remove(edgeToRemove)
                self.__edgeCounter -= 1
            for edge in vertexToRemove.incomingEdges():
                self.__edges.remove(edge)
                self.__edgeCounter -= 1
            self.__vertices.remove(vertexToRemove)
            self.__vertexCounter -= 1

    def printGraph(self):
        for vertex in self.__vertices:
            vertex.printVertex()

Bit of test code to explore the graph

In [5]:
myGraph = Graph(1)
root = myGraph.createRoot(1)
myGraph.createChildVertex(root, 2)
firstChild = myGraph.createChildVertex(root, 2)
secondChild = myGraph.createChildVertex(firstChild, 1)
myGraph.createChildVertex(secondChild, 1)
myGraph.printGraph()

# remove second child
myGraph.removeVertex(secondChild)
print("\n")
myGraph.printGraph()
bob=1
# foo

vertex id = 1, type = 1, incoming = 0, outgoing = 2
vertex id = 2, type = 2, incoming = 1, outgoing = 0
vertex id = 3, type = 2, incoming = 1, outgoing = 1
vertex id = 4, type = 1, incoming = 1, outgoing = 1
vertex id = 5, type = 1, incoming = 1, outgoing = 0


vertex id = 1, type = 1, incoming = 0, outgoing = 2
vertex id = 2, type = 2, incoming = 1, outgoing = 0
vertex id = 3, type = 2, incoming = 1, outgoing = 1
vertex id = 5, type = 1, incoming = 1, outgoing = 0
